In [ ]:
import numpy as np
import pandas as pd
import requests
import json
import seaborn as sns
import re
import matplotlib.pyplot as plt

%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Скрипт, целью которого является сбор данных через API NY Times заголовков статей и выдержек за временной промежуток с 1920 по 2020 год:

In [ ]:
df_bag_of_sentence = {}

for year in range(1921, 2021):
    bag_of_sentence = []
    for month in range(1, 13):
        articles_request = requests.get(f'https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key=kqUfZjUcBrYxAuOcQFteCDFAGCbeMMJm')
        if articles_request.ok:
            articles_content = json.loads(articles_request.text)
            number_of_news_items = len(articles_content['response']['docs'])

            for i in range(number_of_news_items):
                if 'abstract' in articles_content['response']['docs'][i].keys():
                    fragment_abstract = articles_content['response']['docs'][i]['abstract']
                    fragment_headline = articles_content['response']['docs'][i]['headline']['main']
                    if fragment_abstract or fragment_headline:
                        fragment = fragment_abstract + ' ' + fragment_headline
                    bag_of_sentence.append(fragment)
                    
    df_bag_of_sentence = pd.DataFrame(data=bag_of_sentence, index=np.full_like(range(len(bag_of_sentence)), year))
    df_bag_of_sentence = df_bag_of_sentence.reset_index()
    df_bag_of_sentence.rename(columns={'index': 'year', 0: 'sentence'}, inplace=True)
    df_bag_of_sentence.to_csv(f'datasets\df_{year}.csv')

In [ ]:
sentence_year = pd.DataFrame()

for year in range(1920, 2021):
    df = pd.read_csv(f'/kaggle/input/nyt-articles-data/df_{year}.csv')
    sentence_year = pd.concat([sentence_year, df])

Получили следующий файл, где Year - это год выхода публикации, а Sentence - это объединенные заголовок и выдержка (изображение ниже):

![](https://clip2net.com/clip/m593032/32ae2-clip-53kb.png?nocache=1)

In [ ]:
sentence_year.drop(columns=['Unnamed: 0'], inplace=True)
sentence_year

In [ ]:
X = sentence_year['sentence']
y = sentence_year['year']

С целью улучшения качества модели было принято решение разделить временной промежуток на 10 частей:

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
y = discretizer.fit_transform(np.array(y).reshape(-1,1))

Посмотрим на распределение таргета (есть небольшой дисбаланс): 

In [ ]:
plt.figure(figsize=(15,8))

sns.distplot(y)

plt.title('Распределение таргета')
plt.xlabel('Временной промежуток')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

Воспользовася TfidfVectorizer для того, чтобы представить текст в векторном виде:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfvec = TfidfVectorizer(min_df = 0.00001, stop_words='english', token_pattern='\\b[A-z][A-z][A-z]+\\b')
tfvec.fit(X_train)

X_train_vec = tfvec.transform(X_train)
X_test_vec = tfvec.transform(X_test)

In [ ]:
X_train_vec.shape, X_test_vec.shape # 0

Обучим предсказательную модель для дискретных величин, где таргетом будет являться временной промежуток (так как данные достаточно разнородны, то ожидать высокого скора модели не приходится, но мы преследуем другую цель):

In [ ]:
from sklearn.naive_bayes import MultinomialNB

y_train = y_train.ravel()
y_test = y_test.ravel()

mnb_clf = MultinomialNB()
mnb_clf.fit(X_train_vec, y_train)

mnb_clf.score(X_train_vec, y_train), mnb_clf.score(X_test_vec, y_test)

Достанем словарь, на котором обучалась наша модель:

In [ ]:
tfvec_voc = np.array(sorted(tfvec.vocabulary_.items(), key=lambda x: x[1]))[:, 0]

Файл **EmotionLookupTable.txt** (http://sentistrength.wlv.ac.uk/ первоначально задумывалось, что воспользуюсь готовым модулем, но он оказался платным) с размеченными словами с позитивным и негативным окрасом, где каждому из слов присуждается численный балл со знаком + или -:

In [ ]:
emotion_words = pd.read_csv('../input/emotion-words/EmotionLookupTable.txt', sep='\t', names=['word', 'score'])
emotion_words

С помощью данного кода берем наиболее важные слова (основанные на коэффициентах в модели) и считаем по ним neg_score и pos_score по файлу emotion_words.

In [ ]:
year_scores = {}

ind_year = mnb_clf.coef_.shape[0]

for i in range(ind_year):

    best_words_by_year = pd.DataFrame(data=np.c_[tfvec_voc, mnb_clf.coef_[i]], columns=['word', 'coeff'])
    best_words_by_year = best_words_by_year.astype({'coeff': 'float64', 'word': 'str'})
    best_words_by_year = best_words_by_year[best_words_by_year.coeff > np.quantile(best_words_by_year.coeff, 0.99)]
    str_best_words_by_year = ' '.join(best_words_by_year['word'].unique())

    neg_score = 0
    pos_score = 0

    for word, score in emotion_words.values:

        if word.endswith('*'):
            match_pattern = re.search(f'[ ]?\\b{word[:-1]}\\w+\\b', str_best_words_by_year)
        else:
            match_pattern = re.search(f'[ ]?\\b{word}\\b', str_best_words_by_year)

        if match_pattern and score < 0:
            match_word = match_pattern.group(0)
            neg_score += abs(score)
            str_best_words_by_year = str_best_words_by_year.replace(match_word, '')
        elif match_pattern and score > 0:
            match_word = match_pattern.group(0)
            pos_score += score
            str_best_words_by_year = str_best_words_by_year.replace(match_word, '')
    
    year_scores[i] = [neg_score, pos_score]

In [ ]:
year_scores

In [ ]:
neg_scores = np.array(list(year_scores.values()))[:, 0]
pos_scores = np.array(list(year_scores.values()))[:, 1]

In [ ]:
res_scores = pd.DataFrame(np.c_[neg_scores / sum(neg_scores) * 100, pos_scores / sum(pos_scores) * 100], columns=['Negative', 'Positive'], index=list(range(1920, 2019, 10)))

In [ ]:
res_scores

Построим итоговый график:

In [ ]:
plt.figure(figsize=(12,6))

plt.plot(res_scores.index, res_scores['Negative'], color='tab:red', label='Негативный')
plt.plot(res_scores.index, res_scores['Positive'], color='tab:blue', label='Позитивный')

xtick_labels = [1920, 1930, 1940, 1950, 1960, 1970, 1980, 1990, 2000, 2010]
plt.xticks(ticks=xtick_labels, rotation=0, fontsize=12, horizontalalignment='center', alpha=.7)
plt.yticks(fontsize=12, alpha=.7)
plt.title("Информационный фон газеты NY Times", fontsize=13)
plt.grid(axis='both', alpha=.3)

plt.gca().spines["top"].set_alpha(0.0)    
plt.gca().spines["bottom"].set_alpha(0.3)
plt.gca().spines["right"].set_alpha(0.0)    
plt.gca().spines["left"].set_alpha(0.3)
plt.legend()
plt.show()

In [ ]:
sentence_year['size_sentence'] = sentence_year['sentence'].apply(lambda x: len(x))

In [ ]:
sentence_year.groupby('year')['size_sentence'].agg(['sum']).plot()